In [1]:
from scvi.dataset import EbiData, MouseAtlas, UnionDataset, AnnDatasetFromAnnData
from Eval_basis import *
import scanpy as sc
import pandas as pd
import scipy.sparse as sparse
from tqdm import tqdm_notebook as tqdm
import urllib

[2019-09-04 15:18:43,673] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [2]:
ebi_datasets_ids = ['E-MTAB-6946', 'E-MTAB-7320',
                    "E-MTAB-7417", "E-MTAB-6173", "E-GEOD-81682", 
                    "E-ENAD-13", "E-GEOD-99235", "E-GEOD-71585", 
                    "E-GEOD-87631", "E-GEOD-90848", "E-MTAB-4547",
                   "E-MTAB-6970", "E-MTAB-5802", "E-MTAB-5661", 
                    "E-GEOD-99058", "E-GEOD-106973", "E-MTAB-7365",
                   "E-MTAB-5553", "E-MTAB-6925", "E-MTAB-6976"]

ebi_data = []
for e_id in tqdm(ebi_datasets_ids, desc="Loading EBI datasets"):
    try:
        data = EbiData("./data", e_id)
    except urllib.error.HTTPError:
        data = EbiData("./data", e_id, result_file='raw')
    ebi_data.append(data)

/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,20) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
[2019-09-04 15:19:55,786] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 15:19:55,791] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 15:19:58,350] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 15:20:01,350] INFO - scvi.dataset.dataset | Downsampled from 167276 to 167276 cells
[2019-09-04 15:20:30,685] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 15:20:30,687] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 15:20:31,728] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 15:20:32,896] INFO - scvi.dataset.dataset | Downsampled from 62848 to 62848 cells
[2019-09-04 

[2019-09-04 15:21:00,939] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 15:21:00,957] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 15:21:00,957] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 15:21:00,960] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 15:21:00,963] INFO - scvi.dataset.dataset | Downsampled from 120 to 120 cells
[2019-09-04 15:21:01,177] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 15:21:01,191] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 15:21:01,192] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 15:21:01,194] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 15:21:01,196] INFO - scvi.dataset.dataset | Downsampled from 87 to 87 cells
[2019-09-04 15:21:01,258] INFO - scvi.data

In [3]:
mouse_ebi = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              low_memory=False)
mouse_ebi.join_datasets(data_source="memory", 
                             data_target="memory", 
                             gene_datasets=ebi_data)
ebi_data = None

Concatenating datasets: 100%|██████████| 20/20 [04:15<00:00,  4.50s/it]
[2019-09-04 15:25:20,075] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 15:25:24,940] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 15:25:24,953] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 15:25:24,957] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 15:25:24,960] INFO - scvi.dataset.dataset | Joined 20 datasets to one of shape 245339 x 22250.


In [4]:
def do_procedure(dataset, ebi_data, model_filename, tsne_filename):
    n_epochs = 100
    colors = None

    print("Training VAE")

    trainer = train_vae(dataset, "./data", f"../trained_models/{model_filename}", n_epochs=n_epochs)

    dot_size = (mpl.rcParams['lines.markersize'] ** 2.0)

    _ = plot_tsne(trainer, trainer.model, dataset, f"./{tsne_filename}", image_datatype="pdf",
                  colors=colors, s=dot_size, edgecolors='black')

    _ = plot_tsne(trainer, trainer.model, ebi_data,
                  f"./plots/ebi_annotated_data_in_{tsne_filename}",
                  image_datatype="pdf", colors=colors, s=dot_size, edgecolors='black')

In [5]:
ebi = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              low_memory=False)
ebi.join_datasets(data_source="memory", 
                             data_target="memory", 
                             gene_datasets=[EbiData("./data")])

[2019-09-04 15:26:10,878] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2019-09-04 15:26:10,898] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 15:26:10,904] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 15:26:11,944] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 15:26:12,988] INFO - scvi.dataset.dataset | Downsampled from 50896 to 35577 cells
Concatenating datasets: 100%|██████████| 1/1 [00:32<00:00, 32.72s/it]
[2019-09-04 15:26:46,157] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 15:26:47,167] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 15:26:47,172] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 15:26:47,173] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annota

GeneExpressionDataset object with n_cells x nb_genes = 35577 x 22250
    gene_attribute_names: 'gene_names'
    cell_attribute_names: 'labels', 'batch_indices', 'local_vars', 'local_means'
    cell_categorical_attribute_names: 'labels', 'batch_indices'

In [7]:
do_procedure(mouse_ebi, ebi, "ebi_data_full", "ebi_data_full_tsne")

Training VAE
Initializing training.
training: 100%|██████████| 100/100 [25:36<00:00, 18.84s/it]
